<a href="https://colab.research.google.com/github/Guptaraj06/Text-Summarization/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
from google.colab import files
files.upload()

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download sunnysai12345/news-summary

In [ ]:
!unzip news-summary.zip

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
df=pd.read_csv('/content/news_summary.csv',encoding='iso-8859-1')
pdf=pd.read_csv('/content/news_summary_more.csv',encoding='iso-8859-1')

In [ ]:
df

In [ ]:
pdf

In [ ]:
pre1=pdf.iloc[:,0:2].copy()

pre2=df.iloc[:,0:6].copy()

pre2['text']=pre2['author'].str.cat(pre2['date'].str.cat(pre2['read_more'].str.cat(pre2['text'].str.cat(pre2['ctext'],
                                                                                                        sep=" "),
                                                                                   sep=" "),
                                                         sep=" "),
                                    sep=" ")


In [ ]:
pre2

In [ ]:
pre=pd.DataFrame()
pre['text']=pd.concat([pre1['text'],pre2['text']],ignore_index=True)
pre['summary']=pd.concat([pre1['headlines'],pre2['headlines']],ignore_index=True)

In [ ]:
pre

In [ ]:
pre.shape

In [ ]:
pre['text'][:10]

In [ ]:
import re

def text_strip(column):
  for row in column:
    row=re.sub("(\\t)",' ',str(row)).lower()

    row=re.sub("(\\r)",' ',str(row)).lower()

    row=re.sub("(\\n)",' ',str(row)).lower()

    row=re.sub("(__+)"," ",str(row)).lower()
    row=re.sub("(--+)"," ",str(row)).lower()
    row=re.sub("(~~+)"," ",str(row)).lower()
    row=re.sub("(\+\++)"," ",str(row)).lower()
    row=re.sub("(\.\.+)"," ",str(row)).lower()

    row=re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]",' ',str(row)).lower()

    row=re.sub('(mailto:)',' ',str(row)).lower()
    row=re.sub(r'(\\x9\d)',' ',str(row)).lower()
    row=re.sub("([iI][nN][cC]\d+)",'INC_NUM',str(row)).lower()
    row=re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)",' ',str(row)).lower()
    row=re.sub('(\.\s+)',' ',str(row)).lower()
    row=re.sub('(\-\s+)',' ',str(row)).lower()
    row=re.sub('(\:\s+)',' ',str(row)).lower()

    row=re.sub("(\s+.\s+)"," ",str(row)).lower()

    try:
      url=re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)',str(row))
      rep_url=url.group(3)
      row=re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)',rep_url,str(row))

    except:
      pass

    row=re.sub("(\s+)"," ",str(row)).lower()

    row=re.sub("(\s+.\s+)"," ",str(row)).lower()


    yield row

In [ ]:
c1=text_strip(pre['text'])
c2=text_strip(pre['summary'])

In [ ]:
from time import time

import spacy


nlp=spacy.load('en_core_web_sm',disable=['ner','parser'])

t=time()

text=[str(doc) for doc in nlp.pipe(c1,batch_size=5000)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))


In [ ]:
t=time()

summary=['_START' + str(doc) +'_END_' for doc in nlp.pipe(c2,batch_size=5000)]

print('time to clean up everything: {}'.format(round((time()-t)/6,0,2)))

In [ ]:
text[0]

In [ ]:
summary[0]

In [ ]:
pre['cleaned_text']=pd.Series(text)
pre['cleaned_summary']=pd.Series(summary)

In [ ]:
text_count=[]
summary_count=[]

In [ ]:
for sent in pre['cleaned_text']:
  text_count.append(len(sent.split()))
for sent in pre['cleaned_summary']:
  summary_count.append(len(sent.split()))

In [ ]:
graph_df=pd.DataFrame()
graph_df['text']= text_count
graph_df['summary']=summary_count

In [ ]:
import matplotlib.pyplot as plt

graph_df.hist(bins=5)
plt.show()

In [ ]:
cnt=0
for i in pre['cleaned_summary']:
  if(len(i.split())<=15):
    cnt=cnt+1
print(cnt/len(pre['cleaned_summary']))

In [ ]:
cnt=0
for i in pre['cleaned_text']:
  if(len(i.split())<=100):
    cnt=cnt+1

print(cnt/len(pre['cleaned_text']))

In [ ]:
max_text_len=100
max_summary_len=15


In [ ]:
cleaned_text=np.array(pre['cleaned_text'])
cleaned_summary=np.array(pre['cleaned_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
  if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
    short_text.append(cleaned_text[i])
    short_summary.append(cleaned_summary[i])

post_pre=pd.DataFrame({'text':short_text,'summary':short_summary})


In [ ]:
post_pre.head(2)

In [ ]:
post_pre['summary']=post_pre['summary'].apply(lambda x:'sostok '+ x +'eostok')

In [ ]:
post_pre.head(2)

**SEQ2SEQ MODEL BUILDING**

In [ ]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(post_pre['text']),np.array(post_pre['summary']),test_size=0.1,random_state=0,shuffle=True)

In [ ]:
import os
os.environ['BACK_END']='tensorflow'

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

x_tokenizer=Tokenizer()
x_tokenizer.fit_on_texts(list(x_tr))

In [ ]:
thresh=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.item():
  tot_cnt=tot_cnt+1
  tot_freq=tot_freq+value
  if(value<thresh):
      cnt=cnt+1
      freq=freq+value

print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)


In [ ]:
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt)
x_tokenizer.fit_on_texts(list(x_tr))

x_tr_seq=x_tokenizer.texts_to_sequences(x_tr)
x_val_seq =x_tokenizer.texts_to_sequences(x_val)

x_tr = pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val= pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

x_voc=x_tokenizer.num_words + 1

print("Size of vocabulary in X = {}".format(x_voc))

In [ ]:
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_tr))


In [ ]:
thresh=6

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value

print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)


In [ ]:
y_tokenizer=Tokenizer(num_words=tot_cnt-cnt)
y_tokenizer.fit_on_texts(list(y_tr))

y_tr_seq=y_tokenizer.texts_to_sequences(y_tr)
y_val_seq=y_tokenizer.texts_to_sequences(y_val)

y_tr=pad_sequences(y_tr_seq,maxlen=max_summary_len,padding='post')
y_val=pad_sequnces(y_val_seq,maxlen=max_summary_len,padding='post')

y_voc=y_tokenizer.num_words+1
print('Size of vocabulary in Y ={}'.format(y_voc))

In [ ]:
ind=[]
for i in range(len(y_tr)):
  cnt=0
  for j in y_tr[i]:
    if(j!=0):
      cnt=cnt+1
  if(cnt==2):
    ind.append(i)

y_tr=np.delete(y_tr,ind,axis=0)
x_tr=np.delete(x_tr,ind,axis=0)

In [ ]:
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

In [ ]:
!pip install gensim

In [ ]:
import gensim
import numpy as np
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input,LSTM,Embedding,Dense,Concatenate,TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option('display.max_colwidth',200)

print('Size of vocabulary from the w2v model={}'.format(x_voc))

latent_dim=300
embedding_dim=200

encoder_inputs=Input(shape=(max_text_len,))

enc_emb=Embedding(x_voc,embedding_dim,trainable=True)(encoder_inputs)

encoder_lstm1=LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1,state_h1,state_c1=encoder_lstm1(enc_emb)

encoder_lstm2=LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2,state_h2,state_c2=encoder_lstm2(encoder_output1)

enocder_lstm3=LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs,state_h,state_c=enocder_lstm3(encoder_output2)

decoder_inputs=Input(shape=(None,))

dec_emb_layer=Embedding(y_voc,embedding_dim,trainable=True)
dec_emb=dec_emb_layer(decoder_inputs)

decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state,decoder_back_state=decoder_lstm(dec_emb,initial_state=[state_h,state_c])

decoder_dense=TimeDistributed(Dense(y_voc,activation='softmax'))
decoder_outputs=decoder_dense(decoder_output)

model=Model([encoder_inputs,decoder_inputs],decoder_ouputs)

model.summary()

In [ ]:
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy')
es=EarlyStopping(monitor='val_loss',patience=2,verbose=1)

In [ ]:
history=model.fit([x_tr,y_tr[:,:-1]],
                  y_tr.reshape(y_tr.shape[0],y_tr.shape[1],1)[:,1:],
                  callbacks=[es],
                  epochs=50,
                  batch_size=128,
                  validation_data=([x_val,y_val[:,:-1]],
                  y_val.reshape(y_val.shape[0],y_val.shape[1],1)[:,1:])
                  )

In [ ]:
from matplotlib import plotly
pyplot.plot(history.history['loss'],label='train')
pyplot.plot(history.history['val_loss'],label='test')
pyplot.legend()
pyplot.show()


In [ ]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [ ]:
enocder_model=Model(inputs=encoder_inputs,outputs=[encoder_outputs,state_h,state_c])

decoder_state_input_h=Input(shape=(latent_dim,))
decoder_state_input_c=Input(shape=(latent_dim,))
decoder_hidden_state_input=Input(shape=(max_text_len,latent_dim))

dec_emb2=dec_emb_layer(decoder_inputs)
decoder_outputs2,state_h2,state_c2=decoder_lstm(dec_emb2,initial_state=[decoder_state_input_h,decoder_state_input_c])

decoder_outputs2=decoder_dense(decoder_outputs2)

decoder_model=Model([decoder_inputs]+
 [decoder_hidden_state_input,decoder_state_input_h,decoder_state_input_c],
 [decoder_outputs2]+[state_h2,state_c2])

In [ ]:
def decode_sequence(input_seq):
  e_out,e_h,e_c=encoder_model.predict(input_seq)

  target_seq=np.zeros((1,1))

  target_seq[0,0]=target_word_index['sostok']

  stop_condition=False
  decoded_sentence=''
  while not stop_condition:
    output_tokens,h,c=decoder_model.predict([target_seq]+[e_out,e_h,e_c])

    sampled_token_index=np.argmax(output_tokens[0,-1,:])

    sampled_token=reverse_target_word_index[sampled_token_index]

    if(sampled_token!='eostok'):
      decoded_sentence+=' '+sampled_token

    if(sampled_token=='eostok' or len(decoded_sentence.split())>=(max_summary_len-1)):
      stop_condition=True

    target_seq=np.zero((1,1))

    target_seq[0,0]=sampled_token_index

    e_h,e_c=h,c

  return decoded_sentence



In [ ]:
def seq2summary(input_seq):
  newString=''
  for i in input_seq:
    if(i!=0 and i!=target_word_index['sostok'] and i!=target_word_index['eostok']):
      newString+=reverse_target_word_index[i]+' '

  return newString

def seq2text(input_seq):
  newString=''
  for i in input_seq:
    if(i!=0):
      newString=newString+reverse_target_word_index[i]+' '

  return newString

In [ ]:
for i in range(0,100):
  print('Review :',seq2text(x_tr[i]))
  print('Original summary :',seq2summary(y_tr[i]))
  print('predicted summary :',decode_sequence(x_tr[i].reshape(1,max_text_len)))
  print('\n')